# Testing clamp_col Function

In [1]:
import torch

def clamp_tensor(tensor: torch.Tensor, bounds: torch.Tensor) -> torch.Tensor:
    """
    Clamps the values of a tensor within given bounds.

    Args:
        tensor (Tensor): The tensor to clamp.
        bounds (Tensor): A tensor containing the lower and upper bounds.

    Returns:
        Tensor: The clamped tensor.
    """
    if bounds.dim() == 2 and bounds.size(0) == 2:
        # Common bounds for all samples: bounds shape is (2, dim)
        lower_bounds, upper_bounds = bounds
    elif bounds.dim() == 3 and bounds.size(1) == 2:
        # Per-sample bounds: bounds shape is (batch_size, 2, dim)
        lower_bounds = bounds[:, 0, :]
        upper_bounds = bounds[:, 1, :]
    else:
        raise ValueError("Invalid bounds dimension. Expected bounds of shape (2, dim) or (batch_size, 2, dim).")
    
    # Perform clamping
    return torch.max(torch.min(tensor, upper_bounds), lower_bounds)

# Original clamp_col function
def clamp_col(t: torch.Tensor, bounds: torch.Tensor) -> torch.Tensor:
    if len(bounds.shape) == 2:
        # Common bounds for all samples
        dim = bounds.shape[-1]
        for i in range(dim):
            t[:, i] = torch.clamp(t[:, i], min=bounds[0][i], max=bounds[1][i])
    else:
        # Per-sample bounds
        assert bounds.shape[0] == t.shape[0], "Batch size of bounds and tensor must match."
        dim = bounds.shape[-1]
        for i in range(dim):
            for j in range(bounds.shape[0]):
                t[j, i] = torch.clamp(t[j, i], min=bounds[j][0][i], max=bounds[j][1][i])
    return t




In [2]:


# Test Case 1: Common bounds for all samples
batch_size = 10
dim = 5
t = torch.randn(batch_size, dim)
bounds = torch.tensor([
    [-1.0, -2.0, -3.0, -4.0, -5.0],  # Lower bounds
    [1.0, 2.0, 3.0, 4.0, 5.0]        # Upper bounds
])


t1 = t.clone()
t2 = t.clone()


output1 = clamp_tensor(t1, bounds)
output2 = clamp_col(t2, bounds)


if torch.allclose(output1, output2):
    print("Test Case 1 Passed: Outputs are identical.")
else:
    print("Test Case 1 Failed: Outputs differ.")
    print("Difference:\n", output1 - output2)


Test Case 1 Passed: Outputs are identical.


In [3]:
# Test Case 2: Per-sample bounds
# Generate per-sample bounds of shape (batch_size, 2, dim)
lower_bounds = torch.linspace(-1, -5, steps=dim).repeat(batch_size, 1)
upper_bounds = torch.linspace(1, 5, steps=dim).repeat(batch_size, 1)
bounds_per_sample = torch.stack([lower_bounds, upper_bounds], dim=1)  # Shape: (batch_size, 2, dim)


t1 = t.clone()
t2 = t.clone()


output1 = clamp_tensor(t1, bounds_per_sample)
output2 = clamp_col(t2, bounds_per_sample)


if torch.allclose(output1, output2):
    print("Test Case 2 Passed: Outputs are identical.")
else:
    print("Test Case 2 Failed: Outputs differ.")
    print("Difference:\n", output1 - output2)



Test Case 2 Passed: Outputs are identical.


# Testing Initial candidates

In [1]:
import torch

# Modified generate_initial_candidates function
def generate_initial_candidates(
    bounds: torch.Tensor, num_candidates: int, random_numbers: torch.Tensor = None
) -> torch.Tensor:
    """
    Generates initial candidate points uniformly within the given bounds.

    Args:
        bounds (Tensor): A tensor containing the lower and upper bounds.
        num_candidates (int): The number of candidates to generate.
        random_numbers (Tensor): Optional tensor of random numbers to use.

    Returns:
        Tensor: A tensor of initial candidate points.
    """
    if bounds.dim() == 2 and bounds.size(0) == 2:
        # Common bounds for all samples
        dim = bounds.shape[1]
        lower_bounds, upper_bounds = bounds
        if random_numbers is None:
            random_numbers = torch.rand(num_candidates, dim)
        initial_points = lower_bounds + (upper_bounds - lower_bounds) * random_numbers
    elif bounds.dim() == 3 and bounds.size(1) == 2:
        # Per-sample bounds
        batch_size = bounds.shape[0]
        dim = bounds.shape[2]
        lower_bounds = bounds[:, 0, :]
        upper_bounds = bounds[:, 1, :]
        if num_candidates != batch_size:
            raise ValueError("For per-sample bounds, num_candidates must be equal to batch_size")
        if random_numbers is None:
            random_numbers = torch.rand(num_candidates, dim)
        initial_points = lower_bounds + (upper_bounds - lower_bounds) * random_numbers
    else:
        raise ValueError("Invalid bounds shape")
    return initial_points





#Not ours
# Modified generate_batch_initial function
def generate_batch_initial(
    bounds: torch.Tensor, batch_size: int, random_numbers: torch.Tensor = None
) -> torch.Tensor:
    """
    Generates initial candidate points uniformly within the given bounds.

    Args:
        bounds (Tensor): A tensor containing the lower and upper bounds.
        batch_size (int): The number of candidates to generate.
        random_numbers (Tensor): Optional tensor of random numbers to use.

    Returns:
        Tensor: A tensor of initial candidate points.
    """
    dim = bounds.shape[-1]
    _initial = torch.zeros((batch_size, dim))
    
    if bounds.dim() == 2:
        # Common bounds for all samples
        if random_numbers is None:
            random_numbers = torch.rand(batch_size, dim)
        for idx in range(dim):
            min_val = bounds[0][idx].item()
            max_val = bounds[1][idx].item()
            _initial[:, idx] = min_val + (max_val - min_val) * random_numbers[:, idx]
    else:
        # Per-sample bounds
        bsz = bounds.shape[0]
        if batch_size != bsz:
            raise ValueError("For per-sample bounds, batch_size must be equal to bounds.shape[0]")
        if random_numbers is None:
            random_numbers = torch.rand(batch_size, dim)
        for idx in range(dim):
            min_vals = bounds[:, 0, idx]
            max_vals = bounds[:, 1, idx]
            _initial[:, idx] = min_vals + (max_vals - min_vals) * random_numbers[:, idx]
    return _initial


In [2]:
#Test case 1 


# Parameters
batch_size = 10
dim = 5
bounds = torch.tensor([
    [-1.0, -2.0, -3.0, -4.0, -5.0],  # Lower bounds
    [1.0, 2.0, 3.0, 4.0, 5.0]        # Upper bounds
])
num_candidates = batch_size

# Generate random numbers
torch.manual_seed(0)
random_numbers = torch.rand(num_candidates, dim)

# Generate initial candidates using both functions
initial_points1 = generate_initial_candidates(bounds, num_candidates, random_numbers)
initial_points2 = generate_batch_initial(bounds, batch_size, random_numbers)

# Compare outputs
if torch.allclose(initial_points1, initial_points2):
    print("Test Case 1 Passed: Outputs are identical.")
else:
    print("Test Case 1 Failed: Outputs differ.")
    print("Difference:\n", initial_points1 - initial_points2)


Test Case 1 Passed: Outputs are identical.


In [3]:
# Test case 2 


# Parameters
batch_size = 10
dim = 5
# Generate per-sample lower and upper bounds
lower_bounds = torch.linspace(-1, -5, steps=dim).unsqueeze(0).repeat(batch_size, 1)
upper_bounds = torch.linspace(1, 5, steps=dim).unsqueeze(0).repeat(batch_size, 1)
bounds_per_sample = torch.stack([lower_bounds, upper_bounds], dim=1)  # Shape: (batch_size, 2, dim)

# Generate random numbers
torch.manual_seed(0)
random_numbers = torch.rand(batch_size, dim)

# Generate initial candidates using both functions
initial_points1 = generate_initial_candidates(bounds_per_sample, batch_size, random_numbers)
initial_points2 = generate_batch_initial(bounds_per_sample, batch_size, random_numbers)

# Compare outputs
if torch.allclose(initial_points1, initial_points2):
    print("Test Case 2 Passed: Outputs are identical.")
else:
    print("Test Case 2 Failed: Outputs differ.")
    print("Difference:\n", initial_points1 - initial_points2)


Test Case 2 Passed: Outputs are identical.


In [4]:
print("Initial Points from generate_initial_candidates:\n", initial_points1)
print("Initial Points from generate_batch_initial:\n", initial_points2)


Initial Points from generate_initial_candidates:
 tensor([[-0.0075,  1.0729, -2.4691, -2.9438, -1.9258],
        [ 0.2682, -0.0396,  2.3787, -0.3550,  1.3231],
        [-0.3022, -0.3931, -2.8660, -2.6491, -2.0611],
        [ 0.0370,  0.7907,  1.8001, -2.7118, -2.1773],
        [ 0.3632,  1.6608, -0.6174,  2.9932, -0.8059],
        [ 0.1058,  1.8110, -2.7830, -2.5182, -1.2658],
        [-0.3898,  1.7280, -1.9445, -1.8413, -3.4932],
        [-0.9366, -1.1675,  2.5788,  1.7849,  2.4234],
        [ 0.0526, -1.0254,  0.5076, -3.7348, -3.6128],
        [-0.5155,  1.2619,  1.7590, -1.7740, -0.1804]])
Initial Points from generate_batch_initial:
 tensor([[-0.0075,  1.0729, -2.4691, -2.9438, -1.9258],
        [ 0.2682, -0.0396,  2.3787, -0.3550,  1.3231],
        [-0.3022, -0.3931, -2.8660, -2.6491, -2.0611],
        [ 0.0370,  0.7907,  1.8001, -2.7118, -2.1773],
        [ 0.3632,  1.6608, -0.6174,  2.9932, -0.8059],
        [ 0.1058,  1.8110, -2.7830, -2.5182, -1.2658],
        [-0.3898,  1.728